In [80]:
import sys
sys.path.append(r'C:\Users\Mi\Desktop\code\git\python-utils-ak')
from utils_ak.interactive_imports import *

def _import_globals(module):
    for x in dir(module):
        if not x.startswith('_'):
            globals()[x] = getattr(module, x)
            
def _load(module_obj, module_name=None, reload=False, import_globals=False):
    module = load(module_obj, module_name, reload, import_globals=False)
    if import_globals:
        _import_globals(module)
    return module

def load_sources():
    _load('src/time.py', import_globals=True, reload=True)
    _load('src/block.py', import_globals=True, reload=True)
    _load('src/color.py', import_globals=True, reload=True)
    _load('src/drawing.py', import_globals=True, reload=True)
    _load('src/interval.py', import_globals=True, reload=True)

In [81]:
load_sources()

# Models

In [82]:
import sys
# sys.path.append('/mnt/6088FEB688FE89B4/Users/arsen/Desktop/code/git/2020.10-umalat/umalat')
sys.path.append(r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat')
from app.models import *
from app.models import Boiling as BoilingModel
from app.models import Termizator as TermizatorModel

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# sqlite_filepath = '/home/akadaner/Desktop/data.sqlite'
sqlite_filepath = r"C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\data.sqlite"
engine = create_engine(f"sqlite:///{sqlite_filepath}")
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [83]:
values = session.query(SKU).all()
skus = pd.DataFrame(values, columns=['sku'])
skus['name'] = skus['sku'].apply(lambda sku:sku.name)
skus['percent'] = skus['sku'].apply(lambda sku:sku.boiling.percent)
skus.head()

,sku,name,percent
0,<SKU 1>,"Моцарелла ""Pretto"" (для бутербродов), 45%, 0,2 кг, т/ф, (9 шт)",2.7
1,<SKU 2>,"Моцарелла ""Pretto"", 45%, 1,2 кг, в/у",2.7
2,<SKU 3>,"Моцарелла ""Unagrande"", 45%, 0,12 кг, ф/п (кубики)",2.7
3,<SKU 4>,"Моцарелла ""Unagrande"", 45%, 3 кг, пл/л",2.7
4,<SKU 5>,"Моцарелла (палочки), 45%, кг, пл/л",2.7


In [84]:
TERMIZATOR = session.query(Termizator).first()
TERMIZATOR.pouring_time = 30

In [85]:
def get_sample_boiling_conf(boiling_type='water'):
    if boiling_type == 'water':
        tmp = skus[skus['percent'] == 3.3]
        sku = tmp.iloc[0]['sku']
        BOILING_CONF = sku.boiling
        BOILING_REQUEST = {sku: 850}
        return BOILING_CONF, BOILING_REQUEST
    elif boiling_type == 'salt':
        tmp = skus[skus['percent'] == 2.7]
        sku = tmp.iloc[0]['sku']
        BOILING_CONF = sku.boiling
        BOILING_REQUEST = {sku: 850}
        return BOILING_CONF, BOILING_REQUEST

In [86]:
def make_melting_and_packing(boiling_conf, boiling_request, boiling_type, 
                            melting_line=None):
    maker = BlockMaker(default_push_func=dummy_push)
    make = maker.make
    
    with make('melting_and_packing'):
        # packing and melting time
        if boiling_type == 'water':
            melting_time = custom_round(850 / boiling_conf.meltings.speed * 60 * 1.3, 5, rounding='ceil')
            packing_time = 0

            # todo: make proper order
            for sku, sku_kg in boiling_request.items():
                packing_speed = min(sku.packing_speed, boiling_conf.meltings.speed)
                packing_time += custom_round(sku_kg / packing_speed * 60, 5, rounding='ceil')
            
        elif boiling_type == 'salt':
            packing_time = 0

            # todo: make proper order
            for sku, sku_kg in boiling_request.items():
                packing_time += custom_round(sku_kg / sku.packing_speed * 60, 5, rounding='ceil')

            melting_time = packing_time
        
        full_melting_time = boiling_conf.meltings.serving_time + melting_time
        
        with make('melting', y=10, time_size=full_melting_time, melting_line=melting_line):
            with make(y=0):
                make('melting_label', time_size=4 * 5)
                make('melting_name', time_size=full_melting_time - 4 * 5)
            with make(y=1):
                make('serving', time_size=boiling_conf.meltings.serving_time)
                make('melting_process', time_size=melting_time, speed=boiling_conf.meltings.speed)

            with make(y=2):
                make(time_size=boiling_conf.meltings.serving_time, visible=False)
                if boiling_type == 'water':
                    make('cooling1', time_size=boiling_conf.meltings.first_cooling_time)
                    make('cooling2', time_size=boiling_conf.meltings.second_cooling_time)
                elif boiling_type == 'salt':
                    make('salting', time_size=boiling_conf.meltings.salting_time)

        # prepare time
        if boiling_type == 'water':
            prepare_time = boiling_conf.meltings.serving_time + boiling_conf.meltings.first_cooling_time + boiling_conf.meltings.second_cooling_time
        elif boiling_type == 'salt':
            prepare_time = boiling_conf.meltings.serving_time + boiling_conf.meltings.salting_time

        with make('packing', t=prepare_time / 5, time_size=packing_time, push_func=add_push):
            with make(y=0):
                make('packing_label', time_size=3 * 5)
                make('packing_name', time_size=packing_time - 3 * 5, name='чильеджина 5кг/сердечки/фдл 0,125(безлактозная)/0,1')
            with make(y=1):
                make('packing_brand', time_size=packing_time, name='безлактозная/претто')

                    # todo: add configurations
        #         with make(y=2):
        #             make('configuration', size=3, visible=False)
        #             make('configuration', size=1)
        #             make('configuration', size=5, visible=False)
        #             make('configuration', size=1)

    res = maker.root.children[0]
    res.rel_props['size'] = max(c.end for c in res.children)
    
    return res

BOILING_CONF, BOILING_REQUEST = get_sample_boiling_conf('salt')
block = make_melting_and_packing(BOILING_CONF, BOILING_REQUEST, 'salt', melting_line='0')
print(draw_print(block))

========================================= melting_and_packing (0,41]
================ melting (0,16]
==== melting_label (0,4]
    ============ melting_name (4,16]
====== serving (0,6]
      ========== melting_process (6,16]
====== block (0,6]
      ========================= salting (6,31]
                               ========== packing (31.0,41.0]
                               === packing_label (0,3]
                                  ======= packing_name (3,10]
                               ========== packing_brand (0,10]



In [87]:
def make_boiling(boiling_conf, boiling_request, boiling_type='water', 
                 name='чильеджина  3,6 альче  8500кг', block_num=12, pouring_line=None, melting_line=None):
    maker = BlockMaker(default_push_func=dummy_push)
    make = maker.make

    with make('boiling', block_num=block_num, boiling_type=boiling_type, name=name):
        # todo: make boiling size

        timings = []
        timings.append(boiling_conf.pourings.pouring_time)
        timings.append(boiling_conf.pourings.soldification_time)
        timings.append(boiling_conf.pourings.cutting_time)
        timings.append(boiling_conf.pourings.pouring_off_time)
        timings.append(boiling_conf.pourings.extra_time)
        
        with make('pouring', time_size=sum(timings), pouring_line=pouring_line):
            with make(y=0):
                make('termizator', time_size=TERMIZATOR.pouring_time)
                make('pouring_name', time_size=sum(timings) - TERMIZATOR.pouring_time)
            with make(y=1):
                make('pouring_and_fermenting', time_size=timings[0])
                make('soldification', time_size=timings[1])
                make('cutting', time_size=timings[2])
                make('pouring_off', time_size=timings[3])
                make('extra', time_size=timings[4])

        # todo: specify parameter
        make('drenator', size=cast_t('04:00'), visible=False)
        
        make(make_melting_and_packing(boiling_conf, boiling_request, boiling_type, melting_line=melting_line))
        

    res = maker.root.children[0]
    res.rel_props['size'] = max(c.end for c in res.children)

    return res

BOILING_CONF, BOILING_REQUEST = get_sample_boiling_conf('salt')
block = make_boiling(BOILING_CONF, BOILING_REQUEST, 'salt', pouring_line='0', melting_line='0')
print(draw_print(block))

==================================================================================================================== boiling (0,116]
=========================== pouring (0,27]
====== termizator (0,6]
      ===================== pouring_name (6,27]
============ pouring_and_fermenting (0,12]
            ====== soldification (12,18]
                  ==== cutting (18,22]
                      === pouring_off (22,25]
                         == extra (25,27]
                           ================================================ drenator (27,75]
                                                                           ========================================= melting_and_packing (75,116]
                                                                           ================ melting (0,16]
                                                                           ==== melting_label (0,4]
                                                                               ============ mel

# Drawing

In [88]:
STYLES = {
    'pouring': {'beg_time': '01:00'},
    'termizator': {'text': '{block_num} налив'},
    'pouring_name': {'text': '{name}'},
    'pouring_and_fermenting': {'text': 'налив/внесение закваски', 'color': 'orange'},
    'soldification': {'text': 'схватка', 'color': 'yellow'},
    'cutting': {'text': 'резка/обсушка', 'color': '#92d050'},
    'pouring_off': {'text': 'слив', 'color': 'red'},
    'melting': {'beg_time': '07:00'},
    'melting_label': {'text': '{block_num}'},
    'melting_name': {'text': '{name}'},
    'serving': {'color': 'orange', 'text': 'подача и вымешивание'},
    'melting_process': {'color': 'orange', 'text': 'плавление/формирование {speed} кг/ч'},
    'cooling1': {'text': 'охлаж1 <{size} * 5> мин'},
    'cooling2': {'text': 'охлаж2 <{size} * 5> мин'},
    'packing': {'beg_time': '07:00'},
    'packing_label': {'text': '{block_num}'},
    'packing_name': {'text': '{name}'},
    'packing_brand': {'color': '#f2dcdb', 'text': '{name}'},
    'configuration': {'color': 'red'},
    'salting': {'color': 'green', 'text': 'посолка'}
}


STYLINGS = {
    'pouring': lambda b: {'y': {'0': 1, '1': 3}[b.abs_props['pouring_line']]} if b.abs_props['boiling_type'] == 'water' else {'y': {'0': 10, '1': 13}[b.abs_props['pouring_line']]},
    'melting': lambda b: {'y': 5} if b.abs_props['boiling_type'] == 'water' else {'y': {'0': 15, '1': 18, '2': 21, '3': 24}[b.abs_props['melting_line']]},
    'packing': lambda b: {'y': 8} if b.abs_props['boiling_type'] == 'water' else {'y': 27},
}

In [104]:
load_sources()

In [105]:
root = Block('root', t=cast_t('01:45'))
BOILING_CONF, BOILING_REQUEST = get_sample_boiling_conf('salt')
block = make_boiling(BOILING_CONF, BOILING_REQUEST, 'salt', melting_line='0', pouring_line='1')
dummy_push(root, block)
draw_schedule(root, STYLES, STYLINGS, 'output.xlsx', init_sheet)

termizator 13 (14,20]
pouring_name 13 (20,41]
pouring_and_fermenting 14 (14,26]
soldification 14 (26,32]
cutting 14 (32,36]
pouring_off 14 (36,39]
extra 14 (39,41]
melting_label 15 (17,21]
melting_name 15 (21,33]
serving 16 (17,23]
melting_process 16 (23,33]
salting 17 (23,48]
packing_label 27 (48.0,51.0]
packing_name 27 (51.0,58.0]
packing_brand 28 (48.0,58.0]


# Schedule

In [161]:
def boiling_validator(parent, boiling):
    if not parent.children:
        return
    b1 = max(parent.children, key=lambda b: b.rel_props.get('t', 0))
    b2 = boiling
    
    b1.rel_props['props_mode'] = 'absolute'
    b2.rel_props['props_mode'] = 'absolute'
    
    b1.upd_abs_props()
    b2.upd_abs_props()
    
    validate_disjoint(b1['pouring'][0]['termizator'], b2['pouring'][0]['termizator'])
    
    if b1.props['boiling_type'] == b2.props['boiling_type']:
        boiling_type = b1.props['boiling_type']
        
        if b1['pouring'].props['pouring_line'] == b2['pouring'].props['pouring_line']:
            validate_disjoint(b1['pouring'], b2['pouring'])
        
        if boiling_type == 'water':
            validate_disjoint(b1['melting_and_packing']['melting'], b2['melting_and_packing']['melting'])
        else:
            if b1['melting_and_packing']['melting'].props['melting_line'] == b2['melting_and_packing']['melting'].props['melting_line']:
                validate_disjoint(b1['melting_and_packing']['melting'], b2['melting_and_packing']['melting'])
            else:
                validate_disjoint(b1['melting_and_packing']['melting'][1]['melting_process'], b2['melting_and_packing']['melting'][1]['melting_process'])
            
        validate_disjoint(b1['melting_and_packing']['packing'], b2['melting_and_packing']['packing'])
        
    b1.rel_props.pop('props_mode')
    b2.rel_props.pop('props_mode')
    
    b1.upd_abs_props()
    b2.upd_abs_props()

In [162]:
from itertools import product

iter_water_props = [{'pouring_line': str(v)} for v in [0, 1]]
iter_salt_props = [{'pouring_line': str(v[0]), 'melting_line': str(v[1])} for v in product([0, 1], [1, 2, 3, 4])]
iter_water_props, iter_salt_props

([{'pouring_line': '0'}, {'pouring_line': '1'}],
 [{'pouring_line': '0', 'melting_line': '1'},
  {'pouring_line': '0', 'melting_line': '2'},
  {'pouring_line': '0', 'melting_line': '3'},
  {'pouring_line': '0', 'melting_line': '4'},
  {'pouring_line': '1', 'melting_line': '1'},
  {'pouring_line': '1', 'melting_line': '2'},
  {'pouring_line': '1', 'melting_line': '3'},
  {'pouring_line': '1', 'melting_line': '4'}])

In [163]:
# draw salt and water
root = Block('root', t=cast_t('01:45'))

boilings = []

def gen_boiling(boiling_type):
    BOILING_CONF, BOILING_REQUEST = get_sample_boiling_conf(boiling_type)
    return make_boiling(BOILING_CONF, BOILING_REQUEST, boiling_type)

for i in range(0):
    boilings.append(gen_boiling('water'))
    
for i in range(2):
    boilings.append(gen_boiling('salt'))

for b in boilings:
    iter_props = iter_water_props if b.props['boiling_type'] == 'water' else iter_salt_props
    dummy_push(root, b, iter_props=iter_props, validator=boiling_validator, beg='last_beg', max_tries=100)

draw_schedule(root, STYLES, STYLINGS, 'output.xlsx', init_sheet)
root

termizator 10 (14,20]
pouring_name 10 (20,41]
pouring_and_fermenting 11 (14,26]
soldification 11 (26,32]
cutting 11 (32,36]
pouring_off 11 (36,39]
extra 11 (39,41]
melting_label 18 (17,21]
melting_name 18 (21,33]
serving 19 (17,23]
melting_process 19 (23,33]
salting 20 (23,48]
packing_label 27 (48.0,51.0]
packing_name 27 (51.0,58.0]
packing_brand 28 (48.0,58.0]
termizator 13 (24,30]
pouring_name 13 (30,51]
pouring_and_fermenting 14 (24,36]
soldification 14 (36,42]
cutting 14 (42,46]
pouring_off 14 (46,49]
extra 14 (49,51]
melting_label 21 (27,31]
melting_name 21 (31,43]
serving 22 (27,33]
melting_process 22 (33,43]
salting 23 (33,58]
packing_label 27 (58.0,61.0]
packing_name 27 (61.0,68.0]
packing_brand 28 (58.0,68.0]


root (21, 21]
  boiling (0, 116]
    pouring (0, 27]
      block (0, 0]
        termizator (0, 6]
        pouring_name (6, 27]
      block (0, 0]
        pouring_and_fermenting (0, 12]
        soldification (12, 18]
        cutting (18, 22]
        pouring_off (22, 25]
        extra (25, 27]
    drenator (27, 75]
    melting_and_packing (75, 116]
      melting (0, 16]
        block (0, 0]
          melting_label (0, 4]
          melting_name (4, 16]
        block (0, 0]
          serving (0, 6]
          melting_process (6, 16]
        block (0, 0]
          block (0, 6]
          salting (6, 31]
      packing (31.0, 41.0]
        block (0, 0]
          packing_label (0, 3]
          packing_name (3, 10]
        block (0, 0]
          packing_brand (0, 10]
  boiling (10, 126]
    pouring (0, 27]
      block (0, 0]
        termizator (0, 6]
        pouring_name (6, 27]
      block (0, 0]
        pouring_and_fermenting (0, 12]
        soldification (12, 18]
        cutting (18, 22]
     

In [142]:
root = Block('root', t=cast_t('01:45'))
b1 = make_boiling(line='0', boiling_conf=BOILING_CONF, boiling_request=BOILING_REQUEST, block_num=12, boiling_type='salt')
b2 = make_boiling(line='1', boiling_conf=BOILING_CONF, boiling_request=BOILING_REQUEST, block_num=13, boiling_type='salt')
dummy_push(root, b1)
dummy_push(root, b2, beg='last_beg', max_tries=200, validator=boiling_validator)
root

TypeError: make_boiling() got an unexpected keyword argument 'line'

In [18]:
draw_schedule(block, STYLES, STYLINGS, 'output.xlsx', init_sheet)

KeyError: 'block_num'

# Final Water Algo

In [33]:
import anyconfig
request = anyconfig.load('2020.11.22 sample_day_request.yaml')
date = cast_datetime(request['Date'])

values = []
for boiling in request['Boilings']:
    for sku_id, sku_kg in boiling['SKUVolumes'].items():
        values.append([boiling['BoilingId'], sku_id, sku_kg])
df = pd.DataFrame(values)

values = []
for boiling_id, boiling_grp in df.groupby(0):
    boiling_dic = boiling_grp[[1,2]].set_index(1).to_dict(orient='index')
    boiling_dic = {k: v[2] for k, v in boiling_dic.items()}  # {'34': 1110.0, '35': 0.0, ...}
    total_kg = sum(boiling_dic.values())
    # round to get full
    # todo: proper logic
    total_kg = custom_round(total_kg, 850, rounding='ceil')
    
    n_boilings = int(total_kg / 850)
    for i in range(n_boilings):
        cur_kg = 850
        boiling_request = {}
        for k, v in list(boiling_dic.items()):
            boil_kg = min(cur_kg, boiling_dic[k])
            
            boiling_dic[k] -= boil_kg
            
            if k not in boiling_request:
                boiling_request[k] = 0
            boiling_request[k] += boil_kg
            
            cur_kg -= boil_kg
            
        if cur_kg != 0:
            # any non-zero
            k = [k for k, v in boiling_request.items() if v != 0][0]
            boiling_request[k] += cur_kg
        boiling_request = {k: v for k, v in boiling_request.items() if v != 0}
        
        # todo: del
        if len(boiling_request.keys()) > 1:
            continue
            
        values.append([boiling_id, boiling_request])
df = pd.DataFrame(values)
df

,0,1
0,1,{'34': 850}
1,1,{'39': 850.0}
2,1,{'39': 850.0}
3,1,{'39': 850.0}
4,1,{'39': 850.0}
5,1,{'39': 850.0}
6,1,{'39': 850.0}
7,1,{'39': 850.0}
8,1,{'4': 850.0}
9,2,{'1': 850}
